# Latitude, Longitude for any pixel in a GeoTiff File
How to generate the latitude and longitude for a pixel at any given position in a GeoTiff file.

In [1]:
from osgeo import ogr, osr, gdal
import reverse_geocoder as rg

In [2]:
# opening the geotiff file
ds = gdal.Open('/Users/macbook/Documents/BTP/Satellite/Data/LC08_L1TP_150041_20171027_20171027_01_RT/LC08_L1TP_150041_20171027_20171027_01_RT_B2.TIF')

In [3]:
col, row, band = ds.RasterXSize, ds.RasterYSize, ds.RasterCount
print(col, row, band)

(7551, 7701, 1)


In [4]:
xoff, a, b, yoff, d, e = ds.GetGeoTransform()
print(xoff, a, b, yoff, d, e)

# details about the params: GDAL affine transform parameters
# xoff,yoff = left corner 
# a,e = weight,height of pixels
# b,d = rotation of the image (zero if image is north up)

(619185.0, 30.0, 0.0, 3151515.0, 0.0, -30.0)


In [5]:
def pixel2coord(x, y):
    """Returns global coordinates from coordinates x,y of the pixel"""
    xp = a * x + b * y + xoff
    yp = d * x + e * y + yoff
    return(xp, yp)

In [6]:
x,y = pixel2coord(col/2,row/2)
print (x, y)

(732435.0, 3036015.0)


#### These global coordinates are in a *projected coordinated system*, which is a representation of the spheroidal earth's surface, but flattened and distorted onto a plane.
#### To convert these into latitude and longitude, we need to convert these coordinates into *geographic coordinate system*.

In [7]:
# get the existing coordinate system
old_cs= osr.SpatialReference()
old_cs.ImportFromWkt(ds.GetProjectionRef())

# create the new coordinate system
wgs84_wkt = """
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.01745329251994328,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]"""
new_cs = osr.SpatialReference()
new_cs.ImportFromWkt(wgs84_wkt)

# create a transform object to convert between coordinate systems
transform = osr.CoordinateTransformation(old_cs,new_cs) 

In [8]:
# converting into geographic coordinate system
lonx, latx, z = transform.TransformPoint(x,y)

In [9]:
print (latx, lonx, z)

(27.42778025378802, 71.351317510464, 0.0)


In [10]:
# rb = ds.GetRasterBand(1)
px,py = col/2,row/2                   # the pixel location
pix = ds.ReadAsArray(px,py,1,1)     
print pix[0][0]                        # pixel value

11647


In [11]:
coordinates = (latx,lonx)
results = rg.search(coordinates)
print results

Loading formatted geocoded file...
[{'name': 'Jaisalmer', 'cc': 'IN', 'lon': '70.90387', 'admin1': 'Rajasthan', 'admin2': 'Jaisalmer', 'lat': '26.91763'}]


In [12]:
print type(results)
print type(results[0])

<type 'list'>
<type 'dict'>


In [13]:
results[0]

{'admin1': 'Rajasthan',
 'admin2': 'Jaisalmer',
 'cc': 'IN',
 'lat': '26.91763',
 'lon': '70.90387',
 'name': 'Jaisalmer'}

In [14]:
results[0]["name"]

'Jaisalmer'

In [15]:
check for many other examples, to make sure it gives
---> name of a District only
---> It matches with te spelling in the crop dataset
---> What if its outside India (on land, on water)
...

In [20]:
k = 10  # If we want k*k pixels in total from the image


for i in range(0,col,col/k):
    for j in range(0,row,row/k):
        
        # fetching the lat and lon coordinates 
        x,y = pixel2coord(i,j)
        lonx, latx, z = transform.TransformPoint(x,y)
        
        # fetching the name of district
        coordinates = (latx,lonx)
        results = rg.search(coordinates)
        
        # The pixel value for that location
        px,py = i,j                   
        pix = ds.ReadAsArray(px,py,1,1)     
        pix = pix[0][0]
        
        # printing
        s = "The pixel value for the location Lat: {0:6.1f}, Long: {1:6.1f} ({2:15}) is {3:7}".format(latx,lonx,results[0]["name"],pix)
        print (s)

The pixel value for the location Lat:   28.5, Long:   70.2 (Sadiqabad      ) is       0
The pixel value for the location Lat:   28.3, Long:   70.2 (Sadiqabad      ) is       0
The pixel value for the location Lat:   28.1, Long:   70.2 (Sadiqabad      ) is       0
The pixel value for the location Lat:   27.9, Long:   70.2 (Sadiqabad      ) is       0
The pixel value for the location Lat:   27.7, Long:   70.2 (Khairpur       ) is       0
The pixel value for the location Lat:   27.4, Long:   70.2 (Khairpur       ) is       0
The pixel value for the location Lat:   27.2, Long:   70.2 (Jaisalmer      ) is       0
The pixel value for the location Lat:   27.0, Long:   70.2 (Jaisalmer      ) is       0
The pixel value for the location Lat:   26.8, Long:   70.2 (Jaisalmer      ) is       0
The pixel value for the location Lat:   26.6, Long:   70.2 (Jaisalmer      ) is       0
The pixel value for the location Lat:   26.4, Long:   70.2 (Jaisalmer      ) is       0
The pixel value for the location

The pixel value for the location Lat:   27.2, Long:   72.0 (Pokaran        ) is   22059
The pixel value for the location Lat:   27.0, Long:   72.0 (Pokaran        ) is   11562
The pixel value for the location Lat:   26.8, Long:   72.0 (Pokaran        ) is   11302
The pixel value for the location Lat:   26.6, Long:   72.0 (Pokaran        ) is   11163
The pixel value for the location Lat:   26.4, Long:   72.0 (Pokaran        ) is       0
The pixel value for the location Lat:   28.5, Long:   72.3 (Yazman         ) is       0
The pixel value for the location Lat:   28.2, Long:   72.3 (Yazman         ) is       0
The pixel value for the location Lat:   28.0, Long:   72.3 (Phalodi        ) is   10845
The pixel value for the location Lat:   27.8, Long:   72.3 (Phalodi        ) is   10621
The pixel value for the location Lat:   27.6, Long:   72.3 (Phalodi        ) is   11299
The pixel value for the location Lat:   27.4, Long:   72.3 (Phalodi        ) is   11236
The pixel value for the location